In [14]:
# in the beginning, we (install and) import all the needed packages
!pip install pandas-datareader
!pip install yfinance
!pip install pyreadr

import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

#import matplotlib.pyplot as plt

  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
  Using cached lxml-4.9.1.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
      Building lxml version 4.9.1.
      Building without Cython.
      Building against libxml2 2.9.4 and libxslt 1.1.29
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-12-arm64-cpython-310
      creating build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/_elementpath.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/sax.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/pyclasslookup.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/__init__.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/builder.py -> build/lib.macosx-12-

Failed to build lxml
  Running setup.py install for lxml ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for lxml did not run successfully.
  │ exit code: 1
  ╰─> [88 lines of output]
      Building lxml version 4.9.1.
      Building without Cython.
      Building against libxml2 2.9.4 and libxslt 1.1.29
      running install
      /opt/homebrew/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib.macosx-12-arm64-cpython-310
      creating build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/_elementpath.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/sax.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/pyclasslookup.py -> build/lib.macosx-12-arm64-cpython-310/lxml

  Using cached yfinance-0.1.87-py2.py3-none-any.whl (29 kB)
  Using cached lxml-4.9.1.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
      Building lxml version 4.9.1.
      Building without Cython.
      Building against libxml2 2.9.4 and libxslt 1.1.29
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-12-arm64-cpython-310
      creating build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/_elementpath.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/sax.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/pyclasslookup.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/__init__.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/builder.py -> build/lib.macosx-12-arm64-

Failed to build lxml
  Running setup.py install for lxml ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for lxml did not run successfully.
  │ exit code: 1
  ╰─> [88 lines of output]
      Building lxml version 4.9.1.
      Building without Cython.
      Building against libxml2 2.9.4 and libxslt 1.1.29
      running install
      /opt/homebrew/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib.macosx-12-arm64-cpython-310
      creating build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/_elementpath.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/sax.py -> build/lib.macosx-12-arm64-cpython-310/lxml
      copying src/lxml/pyclasslookup.py -> build/lib.macosx-12-arm64-cpython-310/lxml


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [15]:
def mkt_cap_scraper(ticker):

    url = 'https://www.macrotrends.net/assets/php/market_cap.php?t=' + ticker
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'html.parser')
    string = str(soup)

    # in this section, we get rid of the unwanted characters in the string
    stripped_begin = string.split('var chartData = [{', 1)[1]
    stripped_end = stripped_begin.split('];\r\n\r', 1)[0]
    stripped_end = re.sub('"date":"','',stripped_end)
    stripped_end = re.sub('},{',';',stripped_end)
    stripped_end = re.sub('","v1"','',stripped_end)
    stripped_end = re.sub('}','',stripped_end)

    stripped = stripped_end.split(';')

    # here we extract the date- and marketcap- parts of string,
    # convert them into desirable type, and save them into a Pandas dataframe

    date, mkt_cap = [], []

    for row in range(len(stripped)):
        datum = stripped[row][:10]
        datum = datetime.strptime(datum, '%Y-%m-%d')
        mc = stripped[row][11:]

        date.append(datum)
        mkt_cap.append(float(mc))

    mkt_cap_tuples = list(zip(date,mkt_cap))
    df_mkt_cap = pd.DataFrame(mkt_cap_tuples, columns=['Date','Mkt_Cap'])

#     df_mkt_cap
    return df_mkt_cap


In [16]:
# loading tickers of selected stocks
tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
tickers = tickers['Ticker']

tickers

0      TER
1      ADM
2     TMUS
3     IDXX
4      LNC
5      WST
6     AMZN
7      DOV
8     ORLY
9      BSX
10     USB
11     ETR
12     AZO
13     PVH
14    BIIB
15     ETN
16     PFE
17     WMB
18    FITB
19    CHRW
Name: Ticker, dtype: object

In [17]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Computing time:', round(elapsed_time,3) , 'seconds.')

mkt_cap_df_list

Execution time: 9.521631956100464 seconds.


[           Date  Mkt_Cap
 0    2009-04-01     0.78
 1    2009-04-02     0.84
 2    2009-04-03     0.89
 3    2009-04-06     0.88
 4    2009-04-07     0.83
 ...         ...      ...
 3439 2022-11-28    14.11
 3440 2022-11-29    13.95
 3441 2022-11-30    14.61
 3442 2022-12-01    14.43
 3443 2022-12-02    14.49
 
 [3444 rows x 2 columns],
            Date  Mkt_Cap
 0    2009-04-01     0.00
 1    2009-04-02     0.00
 2    2009-04-03     0.00
 3    2009-04-06     0.00
 4    2009-04-07     0.00
 ...         ...      ...
 3439 2022-11-28    54.00
 3440 2022-11-29    54.11
 3441 2022-11-30    54.70
 3442 2022-12-01    51.23
 3443 2022-12-02    51.45
 
 [3444 rows x 2 columns],
            Date  Mkt_Cap
 0    2009-04-01     6.24
 1    2009-04-02     6.60
 2    2009-04-03     6.45
 3    2009-04-06     6.32
 4    2009-04-07     5.95
 ...         ...      ...
 3439 2022-11-28   187.96
 3440 2022-11-29   186.35
 3441 2022-11-30   189.91
 3442 2022-12-01   190.35
 3443 2022-12-02   191.10
 
 [3444

In [ ]:
df = pd.concat(list_of_dataframes)

In [ ]:
tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
# tickers.head()

tickers = tickers.Ticker
# tickers.head()

long_names = pd.DataFrame(index = range(len(tickers)), columns = 2)

for i in range(len(tickers)):
    long_names[i,0] = tickers[i]
    header = str(curlGetHeaders("https://www.macrotrends.net/stocks/charts/" + tickers[i])[5])
    long_names[i,1] = "Location: https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + header
    long_names[i,1] = gsub("/\r\n","",long_names[i,2])
    
    
long_names

#stocks <- pd.Data.Frame()

In [2]:
# url = https://www.macrotrends.net/stocks/charts/AAPL/apple/price-book
#page = requests.get('https://www.macrotrends.net/stocks/charts/AAPL')#/apple/price-book')


In [ ]:
https://www.macrotrends.net/assets/php/market_cap.php?t=AAPL

In [ ]:
get.packages('rvest')
library(rvest)

stocks <- pd.Data.Frame(read.csv("tickers_set.csv", sep=","))
colnames(stocks) <- c("Index","Ticker") #maybe pridat adresu
head(stocks,3)

stock_names <- rep(NA,length(stocks$Ticker))
for (i in seq(1:length(stocks$Ticker))){
  stock_names[i] <- as.character(stocks[i,2])
}

long_names <- data.frame(matrix(NA, nrow = length(stocks$Ticker), ncol = 2))
for (i in seq(1:length(stocks$Ticker))){
  long_names[i,1] <- stock_names[i]
  long_names[i,2] <- gsub(paste0("Location: https://www.macrotrends.net/stocks/charts/", stock_names[i],"/"),"",as.character(curlGetHeaders(paste0("https://www.macrotrends.net/stocks/charts/",stock_names[i]))[5]))
  long_names[i,2] <- gsub("/\r\n","",long_names[i,2])
}
names(long_names) <- c("Ticker", "Long_Name")
print(long_names)
write.csv(long_names,"long_names.csv")


1-Gathering the data [Daniel, Parsa] 

2-Cleaning and building a data set (readable one for the regression function) [Daniel, Parsa] 

In [ ]:
from datetime import date
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf

yf.pdr_override() 

today = date.today()
start_date= “2017–01–01”
end_date=”2019–11–30"